In [3]:
from __future__ import division
from __future__ import print_function

import os
import glob
import time
import random
import argparse
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import scipy.sparse as sp
import torch


In [4]:
def encode_onehot(labels):
    classes = set(labels)
    # identity创建方矩阵
    # 字典key为label的值，value为矩阵的每一行
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in enumerate(classes)}
    # get函数得到字典key对应的value
    labels_onehot = np.array(list(map(classes_dict.get, labels)), dtype=np.int32)
    return labels_onehot

    # map() 会根据提供的函数对指定序列做映射
    # 第一个参数 function 以参数序列中的每一个元素调用 function 函数，返回包含每次 function 函数返回值的新列表
    #  map(lambda x: x ** 2, [1, 2, 3, 4, 5])
    #  output:[1, 4, 9, 16, 25]

In [5]:
def load_data(path="./data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))
    
    # content file的每一行的格式为: <paper_id> <word_attributes> <class_label>
    # 分别对应 0, 1:-1, -1
    # feature为第二列到倒数第二列，labels为最后一列
    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset), dtype=np.dtype(str))

    # 储存为csr型稀疏矩阵
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    # cites file的每一行格式为: <cited paper ID>  <citing paper ID>
    # 根据前面的contents与这里的cites创建图，算出edges矩阵与adj矩阵
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    
    # 由于文件中节点并非是按顺序排列的，因此建立一个编号为0-(node_size-1)的哈希表idx_map，
    # 哈希表中每一项为old id: number，即节点id对应的编号为number
    idx_map = {j: i for i, j in enumerate(idx)}
    
    # edges_unordered为直接从边表文件中直接读取的结果，是一个(edge_num, 2)的数组，每一行表示一条边两个端点的idx
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset), dtype=np.int32)
    
    # flatten：降维，返回一维数组
    # 边的edges_unordered中存储的是端点id，要将每一项的old id换成编号number
    # 在idx_map中以idx作为键查找得到对应节点的编号，reshape成与edges_unordered形状一样的数组
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())), dtype=np.int32).reshape(edges_unordered.shape)
    
    # 根据coo矩阵性质，这一段的作用就是，网络有多少条边，邻接矩阵就有多少个1，
    # 所以先创建一个长度为edge_num的全1数组，每个1的填充位置就是一条边中两个端点的编号，
    # 即edges[:, 0], edges[:, 1]，矩阵的形状为(node_size, node_size)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])), shape=(labels.shape[0], labels.shape[0]), dtype=np.float32)

    # build symmetric adjacency matrix
    # 对于无向图，邻接矩阵是对称的。上一步得到的adj是按有向图构建的，转换成无向图的邻接矩阵需要扩充成对称矩阵
    # 将i->j与j->i中权重最大的那个, 作为无向图的节点i与节点j的边权.
    # https://blog.csdn.net/Eric_1993/article/details/102907104
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize_features(features)
    
    # eye创建单位矩阵，第一个参数为行数，第二个为列数
    # 论文里A^=(D~)^0.5 A~ (D~)^0.5这个公式
    # 对应公式A~=A+I_N
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    
    # 分别构建训练集、验证集、测试集，并创建特征矩阵、标签向量和邻接矩阵的tensor，用来做模型的输入
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    adj = torch.FloatTensor(np.array(adj.todense()))
    features = torch.FloatTensor(np.array(features.todense()))
    # 邻接矩阵转为tensor处理
    labels = torch.LongTensor(np.where(labels)[1])

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [6]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    # https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780
    # https://towardsdatascience.com/understanding-graph-convolutional-networks-for-node-classification-a2bfdb7aba7b
    # 论文里A^=(D~)^0.5 A~ (D~)^0.5这个公式
    # 对每一行求和
    rowsum = np.array(mx.sum(1))
    # (D~)^0.5
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    # 如果某一行全为0，则r_inv算出来会等于无穷大，将这些行的r_inv置为0
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    # 构建对角元素为r_inv的对角矩阵
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    # 论文里A^=(D~)^0.5 A~ (D~)^0.5这个公式
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

In [7]:
def normalize_features(mx):
    """Row-normalize sparse matrix"""
    # 对每一行求和
    rowsum = np.array(mx.sum(1))
    # 求倒数
    r_inv = np.power(rowsum, -1).flatten()
    # 如果某一行全为0，则r_inv算出来会等于无穷大，将这些行的r_inv置为0
    r_inv[np.isinf(r_inv)] = 0.
    # 构建对角元素为r_inv的对角矩阵
    r_mat_inv = sp.diags(r_inv)
    # 用对角矩阵与原始矩阵的点积起到标准化的作用，原始矩阵中每一行元素都会与对应的r_inv相乘，最终相当于除以了sum
    mx = r_mat_inv.dot(mx)
    return mx

In [8]:
def accuracy(output, labels):
    # 使用type_as(tesnor)将张量转换为给定类型的张量
    preds = output.max(1)[1].type_as(labels)
    # 记录等于preds的label eq:equal
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [9]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [10]:
class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        # 学习因子
        self.alpha = alpha
        self.concat = concat
        
        # 建立都是0的矩阵，大小为（输入维度，输出维度）
        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        # xavier初始化
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        # 这里的self.a,对应的是论文里的向量a，故其维度大小应该为(2*out_features, 1)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        # h.shape: (N, in_features), Wh.shape: (N, out_features)
        # h.shape: torch.Size([2708, 8]) 8是label的个数
        Wh = torch.mm(h, self.W)
        a_input = self._prepare_attentional_mechanism_input(Wh)
        
        # 即论文里的eij
        # squeeze除去维数为1的维度
        # [2708, 2708, 16]与[16, 1]相乘再除去维数为1的维度，故其维度为[2708,2708],与领接矩阵adj的维度一样
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        
        # mask-attention
        
        # 维度大小与e相同，所有元素都是-9*10的15次方
        zero_vec = -9e15*torch.ones_like(e)
        
        # 故adj的领接矩阵的大小为[2708, 2708] (归一化处理之后的)
        # 故当adj>0，即两结点有边，则用gat构建的矩阵e，若adj=0,则另其为一个很大的负数，这么做的原因是进行softmax时，这些数就会接近于0了
        attention = torch.where(adj > 0, e, zero_vec)
        
        # 对应论文公式3，attention就是公式里的a_ij
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        N = Wh.size()[0] # number of nodes

        # Below, two matrices are created that contain embeddings in their rows in different orders.
        # (e stands for embedding)
        # These are the rows of the first matrix (Wh_repeated_in_chunks): 
        # e1, e1, ..., e1,            e2, e2, ..., e2,            ..., eN, eN, ..., eN
        # '-------------' -> N times  '-------------' -> N times       '-------------' -> N times
        
        # 
        # These are the rows of the second matrix (Wh_repeated_alternating): 
        # e1, e2, ..., eN, e1, e2, ..., eN, ..., e1, e2, ..., eN 
        # '----------------------------------------------------' -> N times
        
        # https://www.jianshu.com/p/a2102492293a
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)
        Wh_repeated_alternating = Wh.repeat(N, 1)
        # Wh_repeated_in_chunks.shape == Wh_repeated_alternating.shape == (N * N, out_features)

        # The all_combination_matrix, created below, will look like this (|| denotes concatenation):
        # e1 || e1
        # e1 || e2
        # e1 || e3
        # ...
        # e1 || eN
        # e2 || e1
        # e2 || e2
        # e2 || e3
        # ...
        # e2 || eN
        # ...
        # eN || e1
        # eN || e2
        # eN || e3
        # ...
        # eN || eN

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        # all_combinations_matrix.shape == (N * N, 2 * out_features)

        return all_combinations_matrix.view(N, N, 2 * self.out_features)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [11]:
class SpecialSpmmFunction(torch.autograd.Function):
    """Special function for only sparse region backpropataion layer."""
    @staticmethod
    def forward(ctx, indices, values, shape, b):
        assert indices.requires_grad == False
        a = torch.sparse_coo_tensor(indices, values, shape)
        ctx.save_for_backward(a, b)
        ctx.N = shape[0]
        return torch.matmul(a, b)

    @staticmethod
    def backward(ctx, grad_output):
        a, b = ctx.saved_tensors
        grad_values = grad_b = None
        if ctx.needs_input_grad[1]:
            grad_a_dense = grad_output.matmul(b.t())
            edge_idx = a._indices()[0, :] * ctx.N + a._indices()[1, :]
            grad_values = grad_a_dense.view(-1)[edge_idx]
        if ctx.needs_input_grad[3]:
            grad_b = a.t().matmul(grad_output)
        return None, grad_values, None, grad_b

In [12]:
class SpecialSpmm(nn.Module):
    def forward(self, indices, values, shape, b):
        return SpecialSpmmFunction.apply(indices, values, shape, b)

In [13]:
class SpGraphAttentionLayer(nn.Module):
    """
    Sparse version GAT layer, similar to https://arxiv.org/abs/1710.10903
    """

    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(SpGraphAttentionLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_normal_(self.W.data, gain=1.414)
                
        self.a = nn.Parameter(torch.zeros(size=(1, 2*out_features)))
        nn.init.xavier_normal_(self.a.data, gain=1.414)

        self.dropout = nn.Dropout(dropout)
        self.leakyrelu = nn.LeakyReLU(self.alpha)
        self.special_spmm = SpecialSpmm()

    def forward(self, input, adj):
        dv = 'cuda' if input.is_cuda else 'cpu'

        N = input.size()[0]
        edge = adj.nonzero().t()

        h = torch.mm(input, self.W)
        # h: N x out
        assert not torch.isnan(h).any()

        # Self-attention on the nodes - Shared attention mechanism
        edge_h = torch.cat((h[edge[0, :], :], h[edge[1, :], :]), dim=1).t()
        # edge: 2*D x E

        edge_e = torch.exp(-self.leakyrelu(self.a.mm(edge_h).squeeze()))
        assert not torch.isnan(edge_e).any()
        # edge_e: E

        e_rowsum = self.special_spmm(edge, edge_e, torch.Size([N, N]), torch.ones(size=(N,1), device=dv))
        # e_rowsum: N x 1

        edge_e = self.dropout(edge_e)
        # edge_e: E

        h_prime = self.special_spmm(edge, edge_e, torch.Size([N, N]), h)
        assert not torch.isnan(h_prime).any()
        # h_prime: N x out
        
        h_prime = h_prime.div(e_rowsum)
        # h_prime: N x out
        assert not torch.isnan(h_prime).any()

        if self.concat:
            # if this layer is not last layer,
            return F.elu(h_prime)
        else:
            # if this layer is last layer,
            return h_prime

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

In [16]:
class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout
        
        # 输入到隐藏层
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)
        
        # multi-head隐藏层到输出
        self.out_att = GraphAttentionLayer(nhid * nheads, nclass, dropout=dropout, alpha=alpha, concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        # 这里的torch.cat即公式（5）中的||
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)

In [17]:
class SpGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Sparse version of GAT."""
        super(SpGAT, self).__init__()
        self.dropout = dropout

        self.attentions = [SpGraphAttentionLayer(nfeat, 
                                                 nhid, 
                                                 dropout=dropout, 
                                                 alpha=alpha, 
                                                 concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = SpGraphAttentionLayer(nhid * nheads, 
                                             nclass, 
                                             dropout=dropout, 
                                             alpha=alpha, 
                                             concat=False)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x, adj))
        return F.log_softmax(x, dim=1)

In [19]:
# Training settings
parser = argparse.ArgumentParser()
# 禁用CUDA训练
parser.add_argument('--no-cuda', action='store_true', default=False, help='Disables CUDA training.')
# 在训练通过期间验证
parser.add_argument('--fastmode', action='store_true', default=False, help='Validate during training pass.')
# sparse GAT选项
parser.add_argument('--sparse', action='store_true', default=False, help='GAT with sparse version or not.')
# 随机种子
parser.add_argument('--seed', type=int, default=72, help='Random seed.')
# 要训练的epoch数
# parser.add_argument('--epochs', type=int, default=10000, help='Number of epochs to train.')
parser.add_argument('--epochs', type=int, default=2, help='Number of epochs to train.')
# 最初的学习率
parser.add_argument('--lr', type=float, default=0.005, help='Initial learning rate.')
# 权重衰减（参数L2损失）
parser.add_argument('--weight_decay', type=float, default=5e-4, help='Weight decay (L2 loss on parameters).')
# 隐藏层单元数量
parser.add_argument('--hidden', type=int, default=8, help='Number of hidden units.')
# 多头注意力机制
parser.add_argument('--nb_heads', type=int, default=8, help='Number of head attentions.')
# dropout率（1-保持概率)
parser.add_argument('--dropout', type=float, default=0.6, help='Dropout rate (1 - keep probability).')
# Leaky ReLU参数
parser.add_argument('--alpha', type=float, default=0.2, help='Alpha for the leaky_relu.')

parser.add_argument('--patience', type=int, default=100, help='Patience')

# args = parser.parse_args()
args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

# 产生随机种子，以使得结果是确定的
random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [21]:
# Model and optimizer
# 模型和优化器

# GAT模型
# nfeat输入单元数，shape[1]表示特征矩阵的维度数（列数）
# nhid中间层单元数量
# nclass输出单元数，即样本标签数=样本标签最大值+1
# dropout参数
# nheads多头注意力数量
sparse = False
if sparse:
    model = SpGAT(nfeat=features.shape[1], 
                nhid=args.hidden, 
                nclass=int(labels.max()) + 1, 
                dropout=args.dropout, 
                nheads=args.nb_heads, 
                alpha=args.alpha)
else:
    model = GAT(nfeat=features.shape[1], 
                nhid=args.hidden, 
                nclass=int(labels.max()) + 1, 
                dropout=args.dropout, 
                nheads=args.nb_heads, 
                alpha=args.alpha)
    
# 构造一个优化器对象Optimizer，用来保存当前的状态，并能够根据计算得到的梯度来更新参数
# Adam优化器
# lr学习率
# weight_decay权重衰减（L2惩罚）
optimizer = optim.Adam(model.parameters(), 
                       lr=args.lr, 
                       weight_decay=args.weight_decay)

# 如果使用GUP则执行这里，数据写入cuda，便于后续加速
cuda = False
if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

features, adj, labels = Variable(features), Variable(adj), Variable(labels)

In [23]:
# 定义训练函数
def train(epoch):
    # 返回当前时间
    t = time.time()
    
    # train的时候使用dropout, 测试的时候不使用dropout
    # pytorch里面eval()固定整个网络参数，没有dropout
    
    # 固定语句，主要针对启用BatchNormalization和Dropout
    model.train()
    
    # 把梯度置零，也就是把loss关于weight的导数变成0
    optimizer.zero_grad()
    # 执行GAT中的forward前向传播
    output = model(features, adj)
    # 最大似然/log似然损失函数，idx_train是140(0~139)
    # nll_loss: negative log likelihood loss
    # https://www.cnblogs.com/marsggbo/p/10401215.html
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    # 准确率
    acc_train = accuracy(output[idx_train], labels[idx_train])
    # 反向传播
    loss_train.backward()
    # 梯度下降，更新值
    optimizer.step()
    
    # Evaluate validation set performance separately,
    # deactivates dropout during validation run.
    # 是否在训练期间进行验证
    if not args.fastmode:
        # 固定语句，主要针对不启用BatchNormalization和Dropout
        model.eval() 
        # 前向传播
        output = model(features, adj)
    
    # 最大似然/log似然损失函数，idx_val是300(200~499)
    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    # 准确率
    acc_val = accuracy(output[idx_val], labels[idx_val])

    # 正在迭代的epoch数
    # 训练集损失函数值
    # 训练集准确率
    # 验证集损失函数值
    # 验证集准确率
    # 运行时间
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.data.item()),
          'acc_train: {:.4f}'.format(acc_train.data.item()),
          'loss_val: {:.4f}'.format(loss_val.data.item()),
          'acc_val: {:.4f}'.format(acc_val.data.item()),
          'time: {:.4f}s'.format(time.time() - t))

    return loss_val.data.item()

In [24]:
# Train model
t_total = time.time()
loss_values = []
bad_counter = 0
best = args.epochs + 1
best_epoch = 0
# epoch数
for epoch in range(args.epochs):
    # 训练
    loss_values.append(train(epoch))

    torch.save(model.state_dict(), '{}.pkl'.format(epoch))
    if loss_values[-1] < best:
        best = loss_values[-1]
        best_epoch = epoch
        bad_counter = 0
    else:
        bad_counter += 1
        
    # 连续patience次数效果不变好则提前停止
    if bad_counter == args.patience:
        break

    files = glob.glob('*.pkl')
    for file in files:
        epoch_nb = int(file.split('.')[0])
        if epoch_nb < best_epoch:
            os.remove(file)

files = glob.glob('*.pkl')
for file in files:
    epoch_nb = int(file.split('.')[0])
    if epoch_nb > best_epoch:
        os.remove(file)

print("Optimization Finished!")
# 已用总时间
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Restore best model
print('Loading {}th epoch'.format(best_epoch))
model.load_state_dict(torch.load('{}.pkl'.format(best_epoch)))

Epoch: 0001 loss_train: 1.9484 acc_train: 0.0857 loss_val: 1.9404 acc_val: 0.2867 time: 4.2909s
Epoch: 0002 loss_train: 1.9341 acc_train: 0.2286 loss_val: 1.9303 acc_val: 0.4333 time: 3.1546s
Optimization Finished!
Total time elapsed: 7.5623s
Loading 1th epoch


<All keys matched successfully>

In [25]:
# 定义测试函数，相当于对已有的模型在测试集上运行对应的loss与accuracy
def compute_test():
    # 固定语句，主要针对不启用BatchNormalization和Dropout
    model.eval()
    # 前向传播
    output = model(features, adj)
    # 最大似然/log似然损失函数，idx_test是1000(500~1499)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    # 准确率
    acc_test = accuracy(output[idx_test], labels[idx_test])
    
#     print("Test set results:",
#           "loss= {:.4f}".format(loss_test.data[0]),
#           "accuracy= {:.4f}".format(acc_test.data[0]))

    # 测试集损失函数值
    # 测试集的准确率
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data),
          "accuracy= {:.4f}".format(acc_test.data))
# Testing
compute_test()

Test set results: loss= 1.9327 accuracy= 0.3430
